In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, os, sys, shutil
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('./icr-identify-age-related-conditions/train.csv')
test = pd.read_csv('./icr-identify-age-related-conditions/test.csv')
meta = pd.read_csv('./icr-identify-age-related-conditions/greeks.csv')

In [3]:
train_init = train.copy()
train['Alpha'] = meta['Alpha']
train['Beta'] = meta['Beta']
train['Gamma'] = meta['Gamma']
train['Delta'] = meta['Delta']

In [4]:
# 处理greeks
# Alpha为A的时候为0，其余为1
train['Alpha'] = train['Alpha'].apply(lambda x: 0 if x == 'A' else 1)

In [5]:
# 将数据集中唯一的离散特征转为01，这可能表明患者的性别
train['EJ'] = train['EJ'].replace({'A': 0, 'B': 1}).astype(float)
test['EJ']  = test['EJ'].replace({'A': 0, 'B': 1}).astype(float)
train_init['EJ'] = train_init['EJ'].replace({'A': 0, 'B': 1}).astype(float)
# 缺失值处理，用中位数填充
train['BQ'].fillna(0, inplace=True)
train.fillna(train.median(), inplace=True)
test.fillna(test.median(), inplace=True)
train_init.fillna(train_init.median(), inplace=True)
# 移除ID列
train_id = train['Id'].copy()
test_id  =  test['Id'].copy()
train = train.drop(['Id'], axis=1)
test  =  test.drop(['Id'], axis=1)

In [6]:
# 将gamma中的M和N都转为0，G，H，E，F，A，B的转为1
train['Gamma'] = train['Gamma'].replace({'M': 0, 'N': 0, 'G': 1, 'H': 1, 'E': 1, 'F': 1, 'A': 1, 'B': 1})
# 将Beta中的C0,B1,A2
train['Beta'] = train['Beta'].replace({'C': 0, 'B': 1, 'A': 2})
# 将Delta中的B0,A1,C1,D2
train['Delta'] = train['Delta'].replace({'B': 0, 'A': 1, 'C': 1, 'D': 2})

# train.head()


In [7]:
# 连续变量归一化处理
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numeric_columns = [_ for _ in train.columns if _ not in ['EJ', 'Alpha', 'Beta', 'Gamma', 'Delta', 'Class']]
train[numeric_columns] = scaler.fit_transform(train[numeric_columns])
test[numeric_columns] = scaler.transform(test[numeric_columns])
# train.head()

In [8]:
# 抛弃高度相关的特征
drop_cols = ['BZ','CL','EH','GL']  # 高度相关
drop_cols2 = ['DY','CB','GB','CH','DL','CU','FS','AZ','GE','EG','EP']  # 无用特征，同一点取得极值
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

In [9]:
# ytrain为离散
ytrain = train[['Class', 'Alpha', 'Beta', 'Gamma', 'Delta']]
train.drop(['Class', 'Alpha', 'Beta', 'Gamma', 'Delta'], axis=1, inplace=True)

In [10]:
# train.head()

# 模型搭建
先直接对Class预测

In [11]:
# 数据集分割
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
x_train, x_test, y_train, y_test = train_test_split(train, ytrain, test_size=0.2, random_state=42)
scale_pos_weight = 4.712962962962963
# print(x_train.shape)
# print(y_train.shape)

In [12]:
def ScoreMetric(ytrue, ypred):
    nc = np.bincount(ytrue);
    return log_loss(ytrue, ypred, sample_weight = 1 / nc[ytrue], eps=1e-15);

In [13]:
from lightgbm import LGBMClassifier

lgb_p1 = {
    'lambda_l1': 0.0004754363672821333,
    'lambda_l2': 1.088904998340126e-06,
    'num_leaves': 142,
    'feature_fraction': 0.8491732535462826,
    'bagging_fraction': 0.8744449358064078,
    'bagging_freq': 1,
    'min_child_samples': 17,
    'learning_rate': 0.03
}# 0.95  0.28-0.24
lgb_p2 = {
    #  'early_stopping_rounds': 116,
    'n_estimators': 8594,
    'learning_rate': 0.05,   #0.2292825799916429
    'num_leaves': 190,
    'max_depth': 3,
    'reg_alpha': 0.3086813444028655,
    'reg_lambda': 0.08439961817618014,
    'bagging_fraction': 1.0,
    'bagging_freq': 1,
    'feature_fraction': 1.0,
    'min_child_samples': 75,
    'colsample_bytree': 0.2572293361418775,
    'lambda_l1': 1.246275770846192e-06,
    'lambda_l2': 0.011660417895786973,
    'subsample': 0.7301110313724658
 } # 0.95   0.20-0.30

lgb1 = LGBMClassifier(**lgb_p1, random_state=42, n_jobs=-1, objective='multiclass', num_class=3)
lgb2 = LGBMClassifier(**lgb_p2, random_state=42, n_jobs=-1, objective='multiclass', num_class=3)

In [14]:
lgb1.fit(x_train, y_train['Beta'])
pred = lgb1.predict(x_test)
acc = accuracy_score(y_test['Beta'], pred)
print('Beta acc:', acc)
# 查看pred中的各个类别的数量
print("1: ", np.sum(y_test['Beta'] == 1), np.sum(pred == 1))
print("2: ", np.sum(y_test['Beta'] == 2), np.sum(pred == 2))
print("0: ", np.sum(y_test['Beta'] == 0), np.sum(pred == 0))

[LightGBM] [Warning] feature_fraction is set=0.8491732535462826, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8491732535462826
[LightGBM] [Warning] lambda_l1 is set=0.0004754363672821333, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004754363672821333
[LightGBM] [Warning] lambda_l2 is set=1.088904998340126e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.088904998340126e-06
[LightGBM] [Warning] bagging_fraction is set=0.8744449358064078, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8744449358064078
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.8491732535462826, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8491732535462826
[LightGBM] [Warning] lambda_l1 is set=0.0004754363672821333, reg_alpha=0.0 wil

In [15]:
from catboost import CatBoost, CatBoostClassifier

cat_p1 = {
    # 'objective': 'CrossEntropy',
    'learning_rate': 0.025,
    'colsample_bylevel': 0.062247778696758224,
    'depth': 7,
    'boosting_type': 'Plain',
    'bootstrap_type': 'MVS'
}# 0.95   0.20-0.36
cat_p2 = {
    # 'objective': 'CrossEntropy',
    'colsample_bylevel': 0.08326847511080138,
    'depth': 3,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bernoulli',
    'learning_rate': 0.20800532359943255,
    'n_estimators': 1389,
    'early_stopping_rounds': 325,
    'subsample': 0.13113402291704018
}# 0.95   0.19-0.32

cat1 = CatBoostClassifier(**cat_p1, random_state=42, loss_function='MultiClass')
cat2 = CatBoostClassifier(**cat_p2, random_state=42, loss_function='MultiClass')

In [16]:
cat1.fit(x_train, y_train['Beta'])
pred = cat1.predict(x_test)
acc = accuracy_score(y_test['Beta'], pred)
print('Beta acc:', acc)
# 查看pred中的各个类别的数量
print(acc)
print("1: ", np.sum(y_test['Beta'] == 1), np.sum(pred == 1))
print("2: ", np.sum(y_test['Beta'] == 2), np.sum(pred == 2))
print("0: ", np.sum(y_test['Beta'] == 0), np.sum(pred == 0))

0:	learn: 1.0826605	total: 148ms	remaining: 2m 27s
1:	learn: 1.0672691	total: 155ms	remaining: 1m 17s
2:	learn: 1.0540582	total: 166ms	remaining: 55.2s
3:	learn: 1.0392802	total: 177ms	remaining: 44.1s
4:	learn: 1.0255789	total: 179ms	remaining: 35.6s
5:	learn: 1.0128960	total: 187ms	remaining: 31s
6:	learn: 0.9995621	total: 194ms	remaining: 27.5s
7:	learn: 0.9860450	total: 199ms	remaining: 24.6s
8:	learn: 0.9747072	total: 207ms	remaining: 22.8s
9:	learn: 0.9638916	total: 217ms	remaining: 21.5s
10:	learn: 0.9546792	total: 226ms	remaining: 20.3s
11:	learn: 0.9438848	total: 235ms	remaining: 19.4s
12:	learn: 0.9337684	total: 244ms	remaining: 18.5s
13:	learn: 0.9237311	total: 253ms	remaining: 17.8s
14:	learn: 0.9148525	total: 263ms	remaining: 17.3s
15:	learn: 0.9045680	total: 273ms	remaining: 16.8s
16:	learn: 0.8959131	total: 283ms	remaining: 16.3s
17:	learn: 0.8837226	total: 286ms	remaining: 15.6s
18:	learn: 0.8752040	total: 295ms	remaining: 15.2s
19:	learn: 0.8668661	total: 303ms	remaini

In [17]:
# # xgb
import xgboost as xgb

xgb_p1 = {
    'lambda': 0.00012146610908121476,
    'alpha': 0.09188910047137025,
    'learning_rate': 0.02,
    'max_depth': 5,
    'min_child_weight': 2,
    'normalize_type': 'forest',
} # 0.18
xgb_p2 = {
 'lambda': 0.8068353722333764,
 'alpha': 1.894027813634802e-08,
 'learning_rate': 0.02,
 'max_depth': 4,
 } # 0.20
 
xgb_p3 = {
 'lambda': 0.0067981943191443815,
 'alpha': 1.1158232780616973e-07,
 'learning_rate': 0.13779959534970157,
 'max_depth': 3,
 } # 0.18 - 0.24      0.94

xgb_p4 = {
 'lambda': 0.07422190240236703,
 'alpha': 7.880746325817019e-07,
 'learning_rate': 0.05647760311555794,
 'max_depth': 3,
 } # 0.95  0.15-0.21

xgb1 = xgb.XGBClassifier(**xgb_p1, random_state=42, n_jobs=-1, objective='multi:softmax', num_class=3)
xgb2 = xgb.XGBClassifier(**xgb_p2, random_state=42, n_jobs=-1, objective='multi:softmax', num_class=3)
xgb3 = xgb.XGBClassifier(**xgb_p3, random_state=42, n_jobs=-1, objective='multi:softmax', num_class=3)
xgb4 = xgb.XGBClassifier(**xgb_p4, random_state=42, n_jobs=-1, objective='multi:softmax', num_class=3)

In [18]:
xgb5 = xgb.XGBClassifier(random_state=42, n_jobs=-1, objective='multi:softmax', num_class=3, learning_rate=0.02, max_depth=4)
xgb2.fit(x_train, y_train['Beta'])
pred = xgb2.predict(x_test)
acc = accuracy_score(y_test['Beta'], pred)
print('Beta acc:', acc)
# 查看pred中的各个类别的数量
print("1: ", np.sum(y_test['Beta'] == 1), np.sum(pred == 1))
print("2: ", np.sum(y_test['Beta'] == 2), np.sum(pred == 2))
print("0: ", np.sum(y_test['Beta'] == 0), np.sum(pred == 0))

Beta acc: 0.8467741935483871
1:  45 43
2:  2 2
0:  77 79


* Stacking Model

In [19]:
from sklearn.ensemble import VotingClassifier
# hard 不支持概率
voting_clf = VotingClassifier(estimators=[('xgb1', xgb1), ('xgb2', xgb2), ('xgb3', xgb3), ('xgb4', xgb4), ('xgb5', xgb5), ('lgb1', lgb1), ('lgb2', lgb2), ('cat1', cat1), ('cat2', cat2)], voting='soft', n_jobs=-1)
voting_clf.fit(x_train, y_train['Beta'])
voting_pred = voting_clf.predict(x_test)
# voting_pred_proba = voting_clf.predict_proba(x_test)
voting_accuracy = accuracy_score(voting_pred, y_test['Beta'])
print('voting_accuracy: ', voting_accuracy)
print("1: ", np.sum(y_test['Beta'] == 1), np.sum(pred == 1))
print("2: ", np.sum(y_test['Beta'] == 2), np.sum(pred == 2))
print("0: ", np.sum(y_test['Beta'] == 0), np.sum(pred == 0))

[LightGBM] [Warning] feature_fraction is set=0.8491732535462826, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8491732535462826
[LightGBM] [Warning] lambda_l1 is set=0.0004754363672821333, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0004754363672821333
[LightGBM] [Warning] lambda_l2 is set=1.088904998340126e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.088904998340126e-06
[LightGBM] [Warning] bagging_fraction is set=0.8744449358064078, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8744449358064078
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=0.2572293361418775 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] lambda_l1 is set=1.246275770846192e-06, reg_alpha=0.3086813444028655 will be ignored. Current value: lambda_l1=1.246275770846192e-06
[LightGBM] [Warning